In [1]:
import os
import pandas as pd

In [15]:
annotations_path = "path_to_dataset_split_into_sentences"
output_path = "output_path"

In [16]:
bin_short = []
bin_medium = []
bin_long = []

for file_name in os.listdir(annotations_path):
    if not file_name.endswith('.ann'):
        continue
    
    annotations = pd.read_csv(f'{annotations_path}/{file_name}', sep='\t', names=['ann_type', 'entity_type', 'text'], encoding='utf-8')
    
    annotations['start_pos'] = annotations['entity_type'].transform(lambda v: int(v.split()[1]))
    annotations['end_pos'] = annotations['entity_type'].transform(lambda v: int(v.split()[2]))
    annotations['entity_type'] = annotations['entity_type'].transform(lambda v: v.split()[0])
    annotations['length'] = annotations.apply(lambda row: row['end_pos'] - row['start_pos'], axis=1)

    file_name_no_ext = file_name.rstrip('.ann')
    
    if not annotations[annotations['length'] > 90].empty:
        bin_long.append(file_name_no_ext)
    elif not annotations[annotations['length'] >38].empty:
        bin_medium.append(file_name_no_ext)
    else:
        bin_short.append(file_name_no_ext)

In [17]:
import random

def split_list(list, percent_to_select = 0.2):
    num_to_select = int(len(list) * percent_to_select)

    random_selection = random.sample(list, num_to_select)
    rest = [element for element in list if element not in random_selection]

    return random_selection, rest

short_test, short_train = split_list(bin_short)
medium_test, medium_train = split_list(bin_medium)
long_test, long_train = split_list(bin_long)

In [18]:
train = short_train + medium_train + long_train
test = short_test + medium_test + long_test

In [19]:
len(train)

2416

In [20]:
len(test)

603

In [21]:
import shutil

for file_name in os.listdir(annotations_path):
    file_name_no_ext = file_name.split('.')[0]

    if file_name_no_ext in test:
        shutil.copy2(f'{annotations_path}/{file_name}', f'{output_path}/test')
    else:
        shutil.copy2(f'{annotations_path}/{file_name}', f'{output_path}/train')      

In [26]:
selected_train_file_names = list(os.listdir(f'{output_path}/train/'))
selected_test_file_names = list(os.listdir(f'{output_path}/test/'))

for file_name in os.listdir(annotations_path):
    if file_name not in selected_train_file_names and file_name not in selected_test_file_names:
        print(file_name)